In [1]:
import numpy as np
import ckwrap
from normals_init import find_neighs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [28]:
all_pts = np.load("xyz_points.npy")
all_norms = np.load("init_normals.npy")

In [55]:
'''creating patches via clustering'''

def process_one_pcd(pts, norms, main_ind):
    dists_ori, indices_ori = find_neighs(pts)
    neighs = pts[indices_ori]
    print("neighs shape: ", neighs.shape)
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)
    print("labels shape: ",labels_per_row.shape)
    for i,( ind, labels) in enumerate(zip(indices_ori, labels_per_row)):
        center = pts[i]
        for clus in np.unique(labels_per_row[0]):
            ind_clus = ind[np.where(labels == clus)]
            print("ind_clus shape: ",ind_clus.shape)
            coords = pts[ind_clus]

            vec2 = np.subtract(center,coords)
            closest_ind = np.argmin(np.linalg.norm(vec2, axis=1))
            closest = coords[closest_ind]
            vec2 = np.delete(coords, closest_ind, axis=0)
            vec1 = np.subtract(center, closest)
            
            crs_pdts = np.cross(vec1, vec2) #check done, works as intented
            print("crs_pdts shape: ",crs_pdts.shape)
            area = np.linalg.norm(crs_pdts, axis=1)
            dot_pdts = np.dot(crs_pdts, norms[i]) #check done, works as intented
            sign = np.divide(dot_pdts, np.abs(dot_pdts), dtype='int')
            print("dot_pdts shape: ",dot_pdts.shape)
            print("area shape: ",area.shape)
            area = np.multiply(area, sign)
            sort_indices = np.argsort(area)
            # area, orient, ind_clus = area[sort_indices], orient[sort_indices], ind_clus[sort_indices]
            # coords_sorted= pts[ind_clus]
            sorted_coords = coords[sort_indices]
            if(sorted_coords.shape[0] < 25):
                sorted_coords = np.concatenate((sorted_coords, np.zeros((25-sorted_coords.shape[0], 3))))
            else :
                sorted_coords = sorted_coords[:25]
            
            # return dot_pdts, area, crs_pdts, coords, sorted_coords
            mat.append(sorted_coords)
        mat = np.array(mat)
        np.save("patches/pcd_"+str(main_ind)+"/pnt_"+str(i)+".npy", mat)
            # break
        break


o, a, cp, co, co_s, co_chk = process_one_pcd(all_pts[0], all_norms[0])

neighs shape:  (100000, 250, 3)
labels shape:  (100000, 250)
ind_clus shape:  (24,)
crs_pdts shape:  (24, 3)
orient shape:  (24,)
area shape:  (24,)


In [61]:
'''creating patches via clustering'''

def process_one_pcd(pts, norms):
    dists_ori, indices_ori = find_neighs(pts)
    neighs = pts[indices_ori]
    labels_per_row = np.apply_along_axis(lambda row: ckwrap.ckmeans(row, 5).labels, 1, dists_ori)
    mat = [[] for _ in np.unique(labels_per_row[0])]
    for i,( ind, labels) in enumerate(zip(indices_ori, labels_per_row)):
        for clus in np.unique(labels_per_row[0]):
            ind_clus = ind[np.where(labels == clus)]
            coords = pts[ind_clus]
            crs_pdts = np.cross(pts[i], coords) 
            orient = np.dot(crs_pdts, norms[i]) 
            area = np.linalg.norm(crs_pdts, axis=1)
            sort_indices = np.argsort(orient)
            coords_sorted = coords[sort_indices]
            mat[clus].append(np.array(coords_sorted, dtype= object))
    return mat

mat = process_one_pcd(all_pts[0], all_norms[0])

In [63]:
len(mat)

5

In [57]:
co

array([[-1.      ,  0.094387, 11.718286],
       [-1.      ,  0.090934, 11.721165],
       [-1.      ,  0.101446, 11.704637],
       [-1.      ,  0.112601, 11.715321],
       [-1.      ,  0.079621, 11.730504],
       [-1.      ,  0.07598 , 11.72498 ],
       [-1.      ,  0.080252, 11.703704],
       [-1.      ,  0.114846, 11.709319],
       [-1.      ,  0.0993  , 11.695321],
       [-1.      ,  0.102208, 11.688072],
       [-1.      ,  0.07869 , 11.748093],
       [-1.      ,  0.064145, 11.735685],
       [-1.      ,  0.087351, 11.752832],
       [-1.      ,  0.058686, 11.714723],
       [-1.      ,  0.08846 , 11.7559  ],
       [-1.      ,  0.068877, 11.747291],
       [-1.      ,  0.063158, 11.694398],
       [-1.      ,  0.133729, 11.716803],
       [-1.      ,  0.066512, 11.748082],
       [-1.      ,  0.132877, 11.697006],
       [-1.      ,  0.128776, 11.690574],
       [-1.      ,  0.048731, 11.720494],
       [-1.      ,  0.066735, 11.75645 ],
       [-1.      ,  0.068291, 11.7

In [49]:
o

array([ 0.        ,  0.0402975 , -0.08238447, -0.21255644,  0.1723234 ,
        0.2148106 ,  0.16494621, -0.23875831, -0.05734563, -0.09128525,
        0.18319691,  0.35292886,  0.08212659,  0.4166241 ,  0.06918626,
        0.29771291,  0.36442621, -0.45911662,  0.32531254, -0.44918388,
       -0.401329  ,  0.53280052,  0.32271438,  0.30455929])

In [50]:
a

array([0.        , 0.04098232, 0.08540148, 0.21451195, 0.17523664,
       0.21721518, 0.16551228, 0.24162554, 0.06418988, 0.09952087,
       0.18976935, 0.35773213, 0.0926781 , 0.41955512, 0.08233837,
       0.30413409, 0.36581471, 0.46283827, 0.33197655, 0.45517517,
       0.40799326, 0.53716679, 0.33100849, 0.31427984])

In [38]:
np.dot(c[20], all_norms[0][0])

-0.4013289978202006

In [6]:
'''checks'''

x = np.array([[10,9,8,7,6],[5,4,3,2,1]])
y = np.array([[0,0,1,2,0],[2,1,2,1,0]])
print(x.shape, y.shape)
# np.where(y==0, axis=1)

(2, 5) (2, 5)


In [25]:
unique_labels = np.unique(y)
label_indices = np.arange(len(y[0]))[:, None]
print(label_indices)
masks = y == 1
print(masks)
#get the x points for each label while maintaining the dimensionality
x0 = np.where(masks, x, np.nan).reshape(x.shape)  
x0 = x0[~np.all(np.isnan(x0), axis=1)].astype(object)

print(x0)
#drop nan from x0
# x0 = x0[~np.isnan(x0)]
# x0

[[0]
 [1]
 [2]
 [3]
 [4]]
[[False False  True False False]
 [False  True False  True False]]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 5